In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Project/2023/BIGCON/

/content/drive/MyDrive/Project/2023/BIGCON


In [3]:
import pandas as pd
import numpy as np

In [7]:
raw_data = pd.read_csv('2023빅콘테스트_빅데이터플랫폼활용분야_지정주제(부동산)_데이터셋/필지단위 소상공인 매출등급 정보.csv', encoding = 'cp949')
match_data = pd.read_csv('필지번호_행정동.csv', encoding = 'cp949')
test_data = pd.read_excel('2023빅콘테스트_빅데이터플랫폼활용분야_지정주제(부동산)_데이터셋/소상공인 매출등급 예상 대상 필지.xlsx')

In [8]:
# test 데이터는 202301과 202302가 열별로 되어있어 각각 데이터프레임 만든 후 raw데이터 밑으로 붙이기
test_2301 = test_data[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2302 = test_data[['LT_UNQ_NO', 'INDUSTRY_CD', 'STDG_EMD_CD']]
test_2301['DATA_CRTR_YM'] = 202301
test_2302['DATA_CRTR_YM'] = 202302

<ipython-input-8-56b6e8fd2302>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2301['DATA_CRTR_YM'] = 202301
<ipython-input-8-56b6e8fd2302>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_2302['DATA_CRTR_YM'] = 202302


In [9]:
test_2301 = test_2301[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]
test_2302 = test_2302[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'INDUSTRY_CD']]

In [10]:
raw_data = pd.concat([raw_data,test_2301, test_2302])

In [12]:
print(raw_data)
print(match_data.head())

      DATA_CRTR_YM            LT_UNQ_NO  STDG_EMD_CD INDUSTRY_CD  SLS_GRD
0           202201  1111010100100030100     11110101         A03      4.0
1           202201  1111010100100480000     11110101         A01      2.0
2           202201  1111010100100590001     11110101         A11      3.0
3           202201  1111010100100590005     11110101         A03      2.0
4           202201  1111010100100690000     11110101         A01      2.0
...            ...                  ...          ...         ...      ...
9995        202302  1130510100100830004     11305101         A03      NaN
9996        202302  1135010300107450000     11350103         A13      NaN
9997        202302  1150010300103400017     11500103         A06      NaN
9998        202302  1144012700116070000     11440127         A03      NaN
9999        202302  1126010400100770063     11260104         A03      NaN

[1176040 rows x 5 columns]
                   PNU  SGG_CD SGG_NM    EMD_CD EMD_NM    ADM_CD ADM_NM
0  111101010

In [15]:
# 신사동이 관악구, 강남구 둘 다에 있어서 처리해줘야함. 이외에 구는 경계에 걸친 애들
for i in range(len(match_data)):
    if match_data['ADM_NM'][i] == '신사동':
        if (match_data['SGG_NM'][i] == '관악구') or (match_data['SGG_NM'][i] == '동작구'):
            match_data['ADM_NM'][i] = '신사동1'
        elif (match_data['SGG_NM'][i] == '강남구') or (match_data['SGG_NM'][i] == '서초구') or (match_data['SGG_NM'][i] == '용산구')or (match_data['SGG_NM'][i] == '성동구'):
            match_data['ADM_NM'][i] = '신사동2'

<ipython-input-15-e2790476f055>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_data['ADM_NM'][i] = '신사동2'
<ipython-input-15-e2790476f055>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_data['ADM_NM'][i] = '신사동1'


raw데이터에서 매칭 안되는 필지번호는 분리

In [16]:
X = pd.DataFrame(raw_data['LT_UNQ_NO'].unique())
Y = pd.DataFrame(match_data['PNU'].unique())
PNU_NOTIN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique())-set(match_data['PNU'].unique()))
PNU_IN_data = pd.DataFrame(set(raw_data['LT_UNQ_NO'].unique()) & set(match_data['PNU'].unique()))
PNU_NOTIN_data.rename(columns={0:'PNU'}, inplace=True)
PNU_IN_data.rename(columns={0:'PNU'}, inplace=True)

In [17]:
print(len(X))
print(len(PNU_IN_data) + len(PNU_NOTIN_data))
print(len(PNU_IN_data))
print(len(PNU_NOTIN_data))

74303
74303
74199
104


In [18]:
raw_in_match = raw_data[raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]
raw_notin_match = raw_data[~raw_data['LT_UNQ_NO'].isin(PNU_IN_data['PNU'])]

In [19]:
print(len(raw_data))
print(len(raw_in_match) + len(raw_notin_match))
print(len(raw_in_match))
print(len(raw_notin_match))

1176040
1176040
1172902
3138


In [21]:
# 데이터에 있는 필지번호와 match를 merge
df = pd.merge(PNU_IN_data, match_data, how='left')

# 중복 필지 중 하나만 남기고 제거
df.drop_duplicates(subset='PNU', keep='first', inplace=True, ignore_index=False)

In [22]:
print(len(PNU_IN_data))
print(len(df))

74199
74199


In [23]:
# raw데이터에 행정동명 합치기
raw_in_match = pd.merge(raw_in_match, df[['PNU', 'ADM_CD', 'ADM_NM']], how='left', left_on = 'LT_UNQ_NO', right_on = 'PNU')
raw_in_match = raw_in_match[['DATA_CRTR_YM', 'LT_UNQ_NO', 'STDG_EMD_CD', 'ADM_CD', 'ADM_NM', 'INDUSTRY_CD', 'SLS_GRD']]
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172902 entries, 0 to 1172901
Data columns (total 7 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
dtypes: float64(1), int64(4), object(2)
memory usage: 71.6+ MB


### raw 데이터에 행정동별 노인인구수 붙이기

일단 만 65세 이상 기준으로 전체, 남, 여 각각 만들었음

In [44]:
# 행정동별 노인인구수
p2201 = pd.read_csv('행정동별_인구수/202201_202201_연령별인구현황_월간.csv', encoding = 'utf-8')
p2202 = pd.read_csv('행정동별_인구수/202202_202202_연령별인구현황_월간.csv', encoding = 'utf-8')
p2203 = pd.read_csv('행정동별_인구수/202203_202203_연령별인구현황_월간.csv', encoding = 'utf-8')
p2204 = pd.read_csv('행정동별_인구수/202204_202204_연령별인구현황_월간.csv', encoding = 'utf-8')
p2205 = pd.read_csv('행정동별_인구수/202205_202205_연령별인구현황_월간.csv', encoding = 'utf-8')
p2206 = pd.read_csv('행정동별_인구수/202206_202206_연령별인구현황_월간.csv', encoding = 'utf-8')
p2207 = pd.read_csv('행정동별_인구수/202207_202207_연령별인구현황_월간.csv', encoding = 'utf-8')
p2208 = pd.read_csv('행정동별_인구수/202208_202208_연령별인구현황_월간.csv', encoding = 'utf-8')
p2209 = pd.read_csv('행정동별_인구수/202209_202209_연령별인구현황_월간.csv', encoding = 'utf-8')
p2210 = pd.read_csv('행정동별_인구수/202210_202210_연령별인구현황_월간.csv', encoding = 'utf-8')
p2211 = pd.read_csv('행정동별_인구수/202211_202211_연령별인구현황_월간.csv', encoding = 'utf-8')
p2212 = pd.read_csv('행정동별_인구수/202212_202212_연령별인구현황_월간.csv', encoding = 'utf-8')
p2301 = pd.read_csv('행정동별_인구수/202301_202301_연령별인구현황_월간.csv', encoding = 'utf-8')
p2302 = pd.read_csv('행정동별_인구수/202302_202302_연령별인구현황_월간.csv', encoding = 'utf-8')

In [52]:
# 신사동1, 신사동2 구분
data_list = [p2201, p2202, p2203, p2204, p2205, p2206, p2207, p2208, p2209, p2210, p2211, p2212, p2301, p2302]

for i in data_list :
  i.iloc[332, 1] = '관악구 신사동1'
  i.iloc[358, 1] = '강남구 신사동2'

"\np2201[p2201['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2201[p2201['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2202[p2202['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2202[p2202['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2203[p2203['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2203[p2203['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2204[p2204['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2204[p2204['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2205[p2205['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2205[p2205['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2206[p2206['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2206[p2206['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2207[p2207['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2207[p2207['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2208[p2208['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2208[p2208['행정기관'] == '강남구 신사동']['행정기관'] = '강남구 신사동2'\n\np2209[p2209['행정기관'] == '관악구 신사동']['행정기관'] = '관악구 신사동1'\np2209[p2209['행정기관'] == '강남구 신

In [58]:
# 행정동별 노인인구수에서 행정구역을 이용해서 행정동 만들기
p2201['ADM_NM'] = p2201['행정기관'].apply(lambda x: x.split()[1])
p2202['ADM_NM'] = p2202['행정기관'].apply(lambda x: x.split()[1])
p2203['ADM_NM'] = p2203['행정기관'].apply(lambda x: x.split()[1])
p2204['ADM_NM'] = p2204['행정기관'].apply(lambda x: x.split()[1])
p2205['ADM_NM'] = p2205['행정기관'].apply(lambda x: x.split()[1])
p2206['ADM_NM'] = p2206['행정기관'].apply(lambda x: x.split()[1])
p2207['ADM_NM'] = p2207['행정기관'].apply(lambda x: x.split()[1])
p2208['ADM_NM'] = p2208['행정기관'].apply(lambda x: x.split()[1])
p2209['ADM_NM'] = p2209['행정기관'].apply(lambda x: x.split()[1])
p2210['ADM_NM'] = p2210['행정기관'].apply(lambda x: x.split()[1])
p2211['ADM_NM'] = p2211['행정기관'].apply(lambda x: x.split()[1])
p2212['ADM_NM'] = p2212['행정기관'].apply(lambda x: x.split()[1])
p2301['ADM_NM'] = p2301['행정기관'].apply(lambda x: x.split()[1])
p2302['ADM_NM'] = p2302['행정기관'].apply(lambda x: x.split()[1])

In [59]:
p2201.head()

,행정기관코드,행정기관,65세이상,65세이상_남,65세이상_여,ADM_NM
0,1111051500,종로구 청운효자동,"2,144",875,"1,269",청운효자동
1,1111053000,종로구 사직동,"1,759",767,992,사직동
2,1111054000,종로구 삼청동,605,259,346,삼청동
3,1111055000,종로구 부암동,"1,799",795,"1,004",부암동
4,1111056000,종로구 평창동,"3,421","1,490","1,931",평창동


In [78]:
# 월별로 merge
raw_2201 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202201], p2201[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2202 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202202], p2202[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2203 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202203], p2203[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2204 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202204], p2204[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2205 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202205], p2205[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2206 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202206], p2206[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2207 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202207], p2207[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2208 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202208], p2208[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2209 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202209], p2209[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2210 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202210], p2210[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2211 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202211], p2211[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2212 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202212], p2212[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2301 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202301], p2301[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')
raw_2302 = pd.merge(raw_in_match[raw_in_match['DATA_CRTR_YM'] == 202302], p2302[['ADM_NM', '65세이상', '65세이상_남', '65세이상_여']], how = 'left', on = 'ADM_NM')

In [82]:
# 월별 데이터 하나로 합치기
raw_in_match = pd.concat([raw_2201, raw_2202, raw_2203, raw_2204, raw_2205, raw_2206, raw_2207, raw_2208, raw_2209,
                          raw_2210, raw_2211, raw_2212, raw_2301, raw_2302])
raw_in_match.reset_index(drop = True, inplace = True)

In [84]:
raw_in_match.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172902 entries, 0 to 1172901
Data columns (total 10 columns):
 #   Column        Non-Null Count    Dtype  
---  ------        --------------    -----  
 0   DATA_CRTR_YM  1172902 non-null  int64  
 1   LT_UNQ_NO     1172902 non-null  int64  
 2   STDG_EMD_CD   1172902 non-null  int64  
 3   ADM_CD        1172902 non-null  int64  
 4   ADM_NM        1172902 non-null  object 
 5   INDUSTRY_CD   1172902 non-null  object 
 6   SLS_GRD       1152932 non-null  float64
 7   65세이상         1172902 non-null  object 
 8   65세이상_남       1172902 non-null  object 
 9   65세이상_여       1172902 non-null  object 
dtypes: float64(1), int64(4), object(5)
memory usage: 89.5+ MB
